In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model
from sqlalchemy import create_engine
%matplotlib inline

import warnings
warnings.filterwarnings(action="ignore")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'


engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

weather_df = pd.read_sql_query('select * from weatherinszeged',con=engine)

engine.dispose()

weather_df.head(10)

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.
5,2006-04-01 03:00:00+00:00,Partly Cloudy,rain,9.222222,7.111111,0.85,13.9587,258.0,14.9569,0.0,1016.66,Partly cloudy throughout the day.
6,2006-04-01 04:00:00+00:00,Partly Cloudy,rain,7.733333,5.522222,0.95,12.3648,259.0,9.9820,0.0,1016.72,Partly cloudy throughout the day.
7,2006-04-01 05:00:00+00:00,Partly Cloudy,rain,8.772222,6.527778,0.89,14.1519,260.0,9.9820,0.0,1016.84,Partly cloudy throughout the day.
8,2006-04-01 06:00:00+00:00,Partly Cloudy,rain,10.822222,10.822222,0.82,11.3183,259.0,9.9820,0.0,1017.37,Partly cloudy throughout the day.
9,2006-04-01 07:00:00+00:00,Partly Cloudy,rain,13.772222,13.772222,0.72,12.5258,279.0,9.9820,0.0,1017.22,Partly cloudy throughout the day.


In [19]:
weather_df['temp diff']=weather_df['apparenttemperature'] - weather_df['temperature']

In [20]:
X= weather_df[['humidity','windspeed']]
Y= weather_df['temp diff']

import statsmodels.api as sm

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              temp diff   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Mon, 19 Aug 2019   Prob (F-statistic):               0.00
Time:                        12:20:17   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.0

In [21]:
#Q: What are the R-squared and adjusted R-squared values? Do you think they are satisfactory? Why?
#A: R-squared and adjusted r-squared values are 0.288.
#They are good values because it's very low.  Only 28.8% of our explanatory variables are able to 
#explain the variance in the outcome variable.

In [22]:
weather_df['humidity_windspeed'] = weather_df['humidity']*weather_df['windspeed']

In [23]:
X= weather_df[['humidity','windspeed', 'humidity_windspeed']]
Y= weather_df['temp diff']

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              temp diff   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Mon, 19 Aug 2019   Prob (F-statistic):               0.00
Time:                        12:20:24   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0839      0

In [24]:
#Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. 
#Now, what is the R-squared of this model? Does this model improve upon the previous one?

#R-squared and adjusted R-squared on 0.341.  Slightly better than the first model.

In [27]:
#Add visibility as an additional explanatory variable to the first model and estimate it. 
#Did R-squared increase? What about adjusted R-squared? 
#Compare the differences put on the table by the interaction term and the visibility in terms of the improvement 
#in the adjusted R-squared. Which one is more useful?

#R-squared and adjusted R-squared did increase compared to the first model.
#However, adding the interaction term between humidity and windspeed showed better r-squared, adjusted r-squared
#values compared to just adding visibility.  So in that sense, the second model is more useful than
#the third model.  

In [26]:
X= weather_df[['humidity','windspeed','visibility']]
Y= weather_df['temp diff']

import statsmodels.api as sm

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              temp diff   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                 1.401e+04
Date:                Mon, 19 Aug 2019   Prob (F-statistic):               0.00
Time:                        12:21:56   Log-Likelihood:            -1.6938e+05
No. Observations:               96453   AIC:                         3.388e+05
Df Residuals:                   96449   BIC:                         3.388e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5756      0.028     56.605      0.0

In [29]:
#AIC, BIC values are as follows for each model:
#3.409e+05
#3.334e+05
#3.388e+05
#even when looking at AIC, BIC values, the second model better than the other two as the values are lower.  